<a href="https://colab.research.google.com/github/JavaFXpert/qiskit4devs-workshop-notebooks/blob/master/multi_qubit_circuits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Multi-qubit quantum circuits
Now we'll explore quantum circuits that contain more than one wire (qubit). The number of computational basis states doubles with each additional wire, so a three-wire circuit contains eight computational basis states, $\vert000\rangle$ through $\vert111\rangle$.

To demonstrate multi-qubit circuits, we'll create a simple quantum circuit with the [Qiskit](https://qiskit.org/) framework. 

In [0]:
#!pip install qiskit
# Include the necessary imports for this program
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute

In this first version of our circuit, we're going to use a simulator in Qiskit, referred to as "statevector_simulator", that will show us the quantum state upon running the circuit. This is not available on a real quantum computer, because we can't observe the quantum state directly; we can only infer it from measurements that collapse the quantum state into one basis state.

In [0]:
# Create a Quantum Register with 3 qubits
qr = QuantumRegister(3)

# Create a Quantum Circuit from the quantum register. Because we're going to use
# the statevector_simulator, we won't measure the circuit or need classical registers.
circ = QuantumCircuit(qr)

# Place an X gate on the 2nd and 3rd wires. The topmost wire is index 0.
circ.x(qr[1])
circ.x(qr[2])

# Draw the circuit
circ.draw(output='mpl')


Now that the quantum circuit has been defined and drawn, let's execute it on a state vector simulator that will give us the quantum state of the circuit.

In [0]:
# Use the BasicAer statevector_simulator backend
from qiskit import BasicAer
backend_sv_sim = BasicAer.get_backend('statevector_simulator')

# Execute the circuit on the state vector simulator
job_sim = execute(circ, backend_sv_sim)

# Grab the results from the job.
result_sim = job_sim.result()

# Obtain the state vector for the quantum circuit
quantum_state = result_sim.get_statevector(circ, decimals=3)

# Output the quantum state vector in a manner that contains a comma-delimited string.
quantum_state

#### The quantum state vector
The output of the previously run cell contains a comma-separated string that represents the circuit's quantum state. Here are a few things to notice about this state vector:

* There are $2^n$ dimensions in the vector (8 in this example), where $n$ is the number of wires in the circuit (3 in this example).
* Each dimension holds the amplitude of that dimension as a [complex number](https://en.wikipedia.org/wiki/Complex_number "Complex number article on Wikipedia").
* The sum of the squared absolute values of these amplitudes is 1

In this example there is a 100% probability that a measurement will result in the state represented by dimension 6 (0 indexed) of this vector. This state is the $\vert110\rangle$ computational basis state, because $110$ is the binary representation of the number $6$. This matches the fact that the qubit states of the circuit are $\vert1\rangle$, $\vert1\rangle$ and $\vert0\rangle$ starting from the bottom of the circuit.

The state vector of this simple circuit may be calculated by taking the [Kronecker product](https://en.wikipedia.org/wiki/Kronecker_product "Kronecker product article on Wikipedia"), also known as tensor product, of its individual qubit states. Beginning from the most-significant qubit (which is on the bottom wire), perform the following calculation:

$$
 \begin{bmatrix}
  0 \\
  1
 \end{bmatrix}\otimes
 \begin{bmatrix}
  0 \\
  1
 \end{bmatrix}\otimes
 \begin{bmatrix}
  1 \\
  0
 \end{bmatrix}=
 \begin{bmatrix}
  0 \\
  0 \\
  0 \\
  0 \\
  0 \\
  0 \\
  1 \\
  0
 \end{bmatrix}
$$

Notice that the result of this calculation and the output of the get_statevector() method are equivalent. We'll now generate a visualization of the quantum state vector on a Q-sphere:

In [0]:
# Plot the state vector on a Q-sphere
from qiskit.tools.visualization import plot_state_qsphere
plot_state_qsphere(quantum_state)

Each line from the center of the sphere to the surface represents a computational basis state, with the opacity of the line reflecting its probability. Notice that there is only one line on this sphere, and you'll soon see that it represents computational basis state $\vert110\rangle$, consistent with the amplitudes in the state vector.

#### Representing the $\vert110\rangle$ state on a Q-sphere
There are several ways to visualize multi-qubit quantum states, one such way being a Q-sphere. The following Q-sphere represents the $\vert110\rangle$ state.
<br/>

<div align='center'><img src='https://raw.githubusercontent.com/JavaFXpert/qiskit4devs-workshop-notebooks/master/images/qsphere-110-state.png' width=400 height=400 title='Q-sphere representation of the |110> state'>
</div>

<br/>
Each of the computational basis states in a quantum state vector are arranged on the surface of a Q-sphere in the following way:

* the all-zeros state, for example $\vert000\rangle$ in a three-qubit circuit, is at the top of the sphere
* the all-ones state, for example $\vert111\rangle$ in a three-qubit circuit, is at the bottom of the sphere
* each latitudinal line contains states that share the same [Hamming weight](https://en.wikipedia.org/wiki/Hamming_weight "Hamming weight article on Wikipedia") (number of ones in its bit string), with Hamming weights increasing moving downward on the sphere.

Other features of this Q-sphere include:

* The opacity of the radial line and volume of the ball associated with a state is proportional to its measurement probability.
* The color of a state's line and ball reflect its _phase_, which will be addressed in future lessons.

The Q-sphere shown here is from the [Quantum Spheres Playground application](https://javafxpert.github.io/quantum-state-spheres/ "Quantum Spheres Playground application"), with which you may visualize multi-qubit quantum states as you interact with their individual Bloch spheres. Also, in the upper left corner of this application is a text field in which you may paste a comma-separated state vector such as the one generated earlier by the simulator. Doing so will cause the Q-sphere to represent the state vector.

Now that we've examined the circuit's quantum state vector, we'll add measurements to the circuit and create an updated drawing.

In [0]:
# Create a Classical Register with 3 bits
cr = ClassicalRegister(3)

# Create the measurement portion of a quantum circuit
meas_circ = QuantumCircuit(qr, cr)

# Create a barrier that separates the gates from the measurements
meas_circ.barrier(qr)

# Measure the qubits into the classical registers
meas_circ.measure(qr, cr)

# Add the measument circuit to the original circuit
complete_circuit = circ + meas_circ

# Draw the new circuit
complete_circuit.draw(output='mpl')


Now we'll execute the updated circuit on a quantum simulator that includes measurements.

In [0]:
# Use the BasicAer qasm_simulator backend
from qiskit import BasicAer
backend_sim = BasicAer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator, running it 1000 times.
job_sim = execute(complete_circuit, backend_sim, shots=1000)

# Grab the results from the job.
result_sim = job_sim.result()

# Print the counts, which are contained in a Python dictionary
counts = result_sim.get_counts(complete_circuit)
print(counts)


The measurements for each shot should be $\vert110\rangle$ because the state vector indicated that the probability for that basis state is $1$. Recall that the square of the absolute value of a state's amplitude is the probability of a measurement resulting in that state. In this example:
$$
 Pr(110) = \left|1\right|^2 = 1
$$
where $Pr$ means probability. We'll now plot a histogram of the results.

In [0]:
# Plot the results on a histogram
from qiskit.tools.visualization import plot_histogram
plot_histogram(counts)

As you may have noticed, this example demonstrates only classical bit manipulation, so go ahead and create an example that leverages quantum mechanical properties. In the following cell, create a three-wire circuit with a Hadamard gate on each wire, and draw the circuit. You won't measure the circuit or need classical registers just yet:

In [0]:
# Include the necessary imports for this program


# Create a Quantum Register with 3 qubits


# Create a Quantum Circuit from the quantum register. Because we're going to use
# the statevector_simulator, we won't measure the circuit or need classical registers.


# Place Hadamard gate on each of the wires.


# Draw the circuit



Now that the quantum circuit has been defined and drawn, execute it on a state vector simulator that calculates the quantum state of the circuit:

In [0]:
# Use the BasicAer statevector_simulator backend


# Execute the circuit on the state vector simulator


# Grab the results from the job.


# Obtain the state vector for the quantum circuit


# Output the quantum state vector in a manner that contains a comma-delimited string.


Generate a visualization of the quantum state vector on a Q-sphere:

In [0]:
# Plot the state vector on a Q-sphere


Copy the comma-delimited string representing the state vector that was output in a previous step. Paste it into the text field in the upper left corner of the [Quantum Spheres Playground application](https://javafxpert.github.io/quantum-state-spheres/ "Quantum Spheres Playground application") to further visualize it in a Q-sphere.

Add measurements to the circuit and create an updated drawing:

In [0]:
# Create a Classical Register with 3 bits


# Create the measurement portion of a quantum circuit


# Create a barrier that separates the gates from the measurements


# Measure the qubits into the classical registers


# Add the measument circuit to the original circuit


# Draw the new circuit


Execute the updated circuit on a quantum simulator that includes measurements:

In [0]:
# Use the BasicAer qasm_simulator backend


# Execute the circuit on the qasm simulator, running it 1000 times.


# Grab the results from the job.


# Print the counts, which are contained in a Python dictionary


Plot a histogram of the results:

In [0]:
# Plot the results on a histogram


The histogram should contain eight bars, with the measurements for each shot distributed fairly evenly among $\vert000\rangle$ through $\vert111\rangle$. The quantum state should be in an equal superposition.

#### Examining this equal superposition state
If the circuit was constructed correctly, the resulting state vector dimensions will all have the same value, namely $\approx0.354$, which is $\frac{1}{\sqrt{8}}$. Therefore, each computational basis state has an equal probability, exactly $\frac{1}{8}$, of being the result of a measurement. Given that there are eight computational basis states, these probabilities add up to $1$ as they should.

This quantum state matches the result of calculating the Kronecker product of the individual qubit states, which are themselves (after encountering a Hadamard) equal superpositions:

$$
 \begin{bmatrix}
  \frac{1}{\sqrt{2}} \\
  \frac{1}{\sqrt{2}}
 \end{bmatrix}\otimes
 \begin{bmatrix}
  \frac{1}{\sqrt{2}} \\
  \frac{1}{\sqrt{2}}
 \end{bmatrix}\otimes
 \begin{bmatrix}
  \frac{1}{\sqrt{2}} \\
  \frac{1}{\sqrt{2}}
 \end{bmatrix}=
 \begin{bmatrix}
  \frac{1}{\sqrt{8}} \\
  \frac{1}{\sqrt{8}} \\
  \frac{1}{\sqrt{8}} \\
  \frac{1}{\sqrt{8}} \\
  \frac{1}{\sqrt{8}} \\
  \frac{1}{\sqrt{8}} \\
  \frac{1}{\sqrt{8}} \\
  \frac{1}{\sqrt{8}}
 \end{bmatrix}
$$

We could represent this state in Dirac notation in the following way:
$$
 \frac{1}{\sqrt{8}}\vert000\rangle+\frac{1}{\sqrt{8}}\vert001\rangle+\frac{1}{\sqrt{8}}\vert010\rangle+\frac{1}{\sqrt{8}}\vert011\rangle+\frac{1}{\sqrt{8}}\vert100\rangle+\frac{1}{\sqrt{8}}\vert101\rangle+\frac{1}{\sqrt{8}}\vert110\rangle+\frac{1}{\sqrt{8}}\vert111\rangle
$$

We could, more succinctly, leverage the [summation](https://en.wikipedia.org/wiki/Summation "Summation on Wikipedia") operator to represent the same state:
$$
 \frac{1}{\sqrt{8}}\sum_{x=0}^{7} \vert{x}\rangle
$$



#### It's your turn to play!
Here's a puzzle for you to solve. Starting with the most recent circuit, change one or more Hadamard gates to X gates in a manner that results in the following quantum state vector:

$$
 \frac{1}{2}\vert001\rangle+
 \frac{1}{2}\vert011\rangle+
 \frac{1}{2}\vert101\rangle+
 \frac{1}{2}\vert111\rangle
$$

To help solve this puzzle or to visualize the solution, experiment with the [Quantum Spheres Playground application](https://javafxpert.github.io/quantum-state-spheres/ "Quantum Spheres Playground application"). For this three-qubit circuit, you'd click individually on the three rightmost Bloch spheres, trying combinations of **X** and **H** gates. Clicking the $\vert0\rangle$ button will reset a qubit, and refreshing the browser will reset the app.

<div align='center'><img src='https://raw.githubusercontent.com/JavaFXpert/qiskit4devs-workshop-notebooks/master/images/quantum-spheres-screen-shot.png' width=600 title='Quantum spheres playground application'></div>
<br/>

You've covered a lot of ground again in this lesson, particularly in the area of multi-qubit circuits. In this the next lesson we'll explore multi-qubit gates, including the mysterious concept of entanglement.